In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


In [2]:
df = pd.read_csv("../data/processed/insurance_feature_engineered.csv")
df.head()


,age,bmi,children,smoker,charges,log_charges,smoker_flag,smoker_bmi_interaction,sex_male,region_northwest,region_southeast,region_southwest,age_group_31-45,age_group_46-60,age_group_60+,bmi_category_obese,bmi_category_overweight,bmi_category_underweight
0,19,27.900,0,yes,16884.92400,9.734176,1,27.9,False,False,False,True,False,False,False,False,True,False
1,18,33.770,1,no,1725.55230,7.453302,0,0.0,True,False,True,False,False,False,False,True,False,False
2,28,33.000,3,no,4449.46200,8.400538,0,0.0,True,False,True,False,False,False,False,True,False,False
3,33,22.705,0,no,21984.47061,9.998092,0,0.0,True,True,False,False,True,False,False,False,False,False
4,32,28.880,0,no,3866.85520,8.260197,0,0.0,True,True,False,False,True,False,False,False,True,False


In [8]:
X = df.drop(columns=["charges", "log_charges"])
X = X.drop(columns=["smoker"])
y = df["log_charges"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate_model(name, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)

    return {
        "model": name,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2
    }



In [13]:
results = []

lin_reg = LinearRegression()
results.append(evaluate_model("LinearRegression", lin_reg, X_train, X_test, y_train, y_test))

results


[{'model': 'LinearRegression',
  'MAE': 0.2607870360267812,
  'RMSE': np.float64(0.4038359086229803),
  'R2': 0.8186229945040895}]

In [14]:
ridge = Ridge(alpha=1.0, random_state=42)
lasso = Lasso(alpha=0.001, random_state=42)

results.append(evaluate_model("Ridge", ridge, X_train, X_test, y_train, y_test))
results.append(evaluate_model("Lasso", lasso, X_train, X_test, y_train, y_test))


In [15]:
rf = RandomForestRegressor(
    n_estimators=400,
    random_state=42,
    n_jobs=-1
)
results.append(evaluate_model("RandomForest", rf, X_train, X_test, y_train, y_test))


In [16]:
gbr = GradientBoostingRegressor(random_state=42)
results.append(evaluate_model("GradientBoosting", gbr, X_train, X_test, y_train, y_test))


In [17]:
results_df = pd.DataFrame(results).sort_values(by="RMSE")
results_df


,model,MAE,RMSE,R2
4,GradientBoosting,0.185027,0.346618,0.866379
3,RandomForest,0.194445,0.372517,0.845665
0,LinearRegression,0.260787,0.403836,0.818623
1,Ridge,0.259862,0.403853,0.818608
2,Lasso,0.256266,0.404897,0.817668


In [18]:
param_grid = {
    "n_estimators": [300, 500],
    "max_depth": [None, 6, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)

grid = GridSearchCV(
    rf_base,
    param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

grid.fit(X_train, y_train)

grid.best_params_, grid.best_score_


({'max_depth': 6,
  'min_samples_leaf': 4,
  'min_samples_split': 10,
  'n_estimators': 300},
 np.float64(-0.3856588214695559))

In [19]:
best_rf = grid.best_estimator_
results.append(evaluate_model("RandomForest_Tuned", best_rf, X_train, X_test, y_train, y_test))

results_df = pd.DataFrame(results).sort_values(by="RMSE")
results_df


,model,MAE,RMSE,R2
4,GradientBoosting,0.185027,0.346618,0.866379
5,RandomForest_Tuned,0.191724,0.351170,0.862847
3,RandomForest,0.194445,0.372517,0.845665
0,LinearRegression,0.260787,0.403836,0.818623
1,Ridge,0.259862,0.403853,0.818608
2,Lasso,0.256266,0.404897,0.817668


In [22]:
import joblib

joblib.dump(gbr, "../models/best_model.joblib")


['../models/best_model.joblib']

A baseline linear regression model was trained on log-transformed medical charges to establish a performance benchmark. Tree-based models (Random Forest and Gradient Boosting) were then evaluated to capture non-linear relationships and interaction effects. Model performance was compared using MAE, RMSE, and R² on a held-out test set, and the best-performing model was optionally tuned via cross-validated hyperparameter search.

Gradient Boosting achieved the best performance across all evaluation metrics, indicating its ability to capture nonlinear relationships and interaction effects inherent in healthcare cost data. While Random Forest models also performed well, boosting provided superior accuracy and explanatory power.

Gradient Boosting was selected as the final model based on superior RMSE and R² on the held-out test set.